In [ ]:
from datascience import *
import numpy as np

%matplotlib inline
import matplotlib.pyplot as plots
plots.style.use('fivethirtyeight')

# Lecture 24 #

## Reviewing Setup from Last Lecture

Recall, we actually had access to the population data.  This is not normally the case.  The methods we are about to study only make sense when you don't have access to the population. 

In [ ]:
sf = Table.read_table('san_francisco_2019.csv')
min_salary = 15 * 20 * 50
sf = sf.where('Salary', are.above(min_salary))

In [ ]:
sf.num_rows

In [ ]:
sf_bins = np.arange(0, 726000, 25000)
sf.hist('Total Compensation', bins=sf_bins)

### Defining the Statistic

Here we are interested in the median (50% percentile) of the total compensation. 

In [ ]:
# Parameter: Median total compensation in the population
def median_comp(t):
    return ... # Implement median of "Total Compensation"

### The Populatin Parameter

We have access to the population so we can compute the **parameter** but in practice we typically won't have access to the population and instead we will have to *estimate* the parameter from a **sample**.

In [ ]:
pop_median = median_comp(sf)
print("Parameter Value:", pop_median)

### The Sample

In practice, the data we would get from most studies would be a sample of the entire population.  This is because collecting a full census is challenging and expensive.

Here we load the (fictional) sample that was taken by Prof. Gonzalez. 

<details><summary>Unimportant and Not True Details</summary>

He collected this sample by selecting 400 names at random from the HR database and then directly asking them how much they make. It took all weekend and he had to give a chocolate to everyone who participated. 
    
    
<details><summary>The Real Truth</summary>
        
Actually, everything above is a complete fabrication.  Prof. Gonzalez simply wrote: 
        
```python
sf.sample(400, with_replacement=False).to_csv("sf_sample.csv")
```
        
</details>
    
</details>

In [ ]:
sample_sf = Table.read_table("sf_sample.csv")
sample_sf.show(4)

In [ ]:
sample_sf.num_rows

### Bootstrap Sampling

We introduced the following function that computes bootstrap samples of the statistic

In [ ]:
def bootstrapper(sample, statistic, num_repetitions):
    """
    Returns the statistic computed on a num_repetitions  
    bootstrap samples from sample.
    """
    bstrap_stats = make_array()
    for i in np.arange(num_repetitions):
        # Step 1: Sample the Sample
        bootstrap_sample = sample.sample()
        # Step 2: compute statistics on the sample of the sample
        bootstrap_stat = statistic(bootstrap_sample)
        # Accumulate the statistics
        bstrap_stats = np.append(bstrap_stats, bootstrap_stat)

    return bstrap_stats    

In [ ]:
bootstrap_medians = bootstrapper(sample_sf, median_comp, 10000)

### Examining the Distribution of the Statistic

When using the boostrap it is important to always examine the distribution of the statistic to ensure that it is symmetric and "bell shaped"

In [ ]:
Table().with_column("bootstraps", bootstrap_medians).hist()
## Extra Viz code

parameter_green = '#32CD32'
plots.ylim(-0.000005, 0.00014)
plots.scatter(pop_median, 0, color=parameter_green, s=40, zorder=2)
plots.title('Bootstrap Medians and the Parameter (Green Dot)');

### Computing the Confidence Interval

We compute the confidence interval for our desired **confidence level* using the followign code:

In [ ]:
def compute_ci(bs_samples, confidence_level):
    """
    Returns the confidence interval for the provided bootstrap samples
    and desired confidence level.
    """
    tail_size = ... # EXERCISE 
    lower = # EXERCISE 
    upper = # EXERCISE 
    return [lower, upper]

In [ ]:
ci = compute_ci(bootstrap_medians, 95)
ci

Visualizing the CI

In [ ]:
Table().with_column("bootstraps", bootstrap_medians).hist()
# cool python trick to deconstruct the array!
[left, right] = ci
# Plotting parameters; you can ignore this code
plots.ylim(-0.000005, 0.00014)
plots.plot(make_array(left, right), make_array(0, 0), color='yellow', lw=3, zorder=1)
plots.scatter(pop_median, 0, color=parameter_green, s=40, zorder=2);

<br><br><br><br>

---

Return to Slides

---

<br><br><br><br>


---

## Simulating the Simulation!

Let's look at what happens if we use the above code repeatedly with separate original samples from the population. How accurate are our Bootstrap Estimates of the Parameter? 

In [ ]:
intervals = Table(['lower', 'upper', 'median', 'good', 'sample_size'])
sample_sizes = [2,4,8,16]
for sample_size in sample_sizes:
    for trial in np.arange(20): # Run 20 trials of each configuration
        # Pay for one new random sample from the population
        og_sample = sf.sample(sample_size, with_replacement=False)
        # Compute the statistic on the sample
        sample_median = median_comp(og_sample)
        # Generate the medians of 1000 bootstrap samples
        bootstrap_medians = bootstrapper(og_sample, median_comp, 1000)
        # Construct the confidence interval
        [ci_lower, ci_upper] = compute_ci(bootstrap_medians, 95)
        # Determine if the confidence interval is good
        is_good = ci_lower <= pop_median <= ci_upper
        # Add a row to the table
        intervals = intervals.with_row(
            [ci_lower, ci_upper, sample_median, is_good, str(sample_size)])

# Add an experiment number
intervals = intervals.with_column("Experiment", np.arange(intervals.num_rows))

In [ ]:
intervals

Here I render a plot of all the confidence intervals with the true value depicted as a solid vertical line.  **This is advanced plotting code and is out of scope for this class.**  But it might be fun to try to understand what it is doing. 

In [ ]:
import plotly.express as px

# Plotly will draw error bars which are the distance in each direction
# from the median
intervals = intervals.with_columns(
        "error_left", intervals.column("median") - intervals.column("lower"),
        "error_right", intervals.column("upper") - intervals.column("median"))

# Making the plot
fig = px.scatter(
    intervals.to_df(), # Plotly requires a pandas dafaframe
    x="median", # X location of my interval center dot
    y="Experiment", # Y location of my interval center dot 
    color="sample_size", # The color to use.
    symbol="good", # The column to use for the symbol
    symbol_map={True: "circle", False: "circle-open"},
    error_x="error_right", # upper error bar size
    error_x_minus="error_left", # lower error bar size
    height=800)
fig.add_vline(pop_median)

<br><br><br><br>

---

Return to Slides

---

<br><br><br><br>


## Confidence Interval for Unknown Population Mean

Now let's look at a more appropriate use of the bootstrap (when we don't have the population).  The baby table from prior lecture had a random sample of moms.

In [ ]:
# Random sample of mother-newborn pairs
births = Table.read_table('baby.csv')
births

What is the average age of mom's who are having children in the entire population of the Bay Area?

**Parameter:** Average age of mom's when they give birth to first child.

**Statistic:** Average age of moms in our sample.

In [ ]:
# Average age of mothers in the sample
np.average(births.column('Maternal Age'))

Remember there was a distribution of ages.

In [ ]:
births.hist('Maternal Age')

We could have also returned the median or even a range of ages as our statistic:

In [ ]:
percentile(50, births.column('Maternal Age'))

Or an interval of 95% of the ages:

In [ ]:
[percentile(2.5, births.column('Maternal Age')),
 percentile(97.5, births.column('Maternal Age'))]

Is this a confidence interval?
<br><br><br><br>

### Compute the Sample Statistic
Since we are interested in estimating the average age of mothers in the population we will use the average age statistic:

In [ ]:
def avg_maternal_age(sample):
    return np.average(sample.column("Maternal Age"))

In [ ]:
sample_statistic = avg_maternal_age(births)
sample_statistic

### Use the Bootstrap to Estimate the CI 

The interval of estimates is the "middle 95%" of the bootstrap estimates.

This is called a *95% confidence interval* for the mean age in the population.


In [ ]:
bootstrap_means = ... # EXERCISE 
avg_maternal_age_ci = ... # EXERCISE 
avg_maternal_age_ci

In [ ]:
resampled_means = Table().with_columns(
    'Bootstrap Sample Mean', bootstrap_means
)
resampled_means.hist(bins=15)
[left, right] = avg_maternal_age_ci
plots.plot([left, right], [0, 0], color='yellow', lw=8);

In [ ]:
births.hist('Maternal Age')
plots.plot([left, right], [0, 0], color='yellow', lw=8);

<br><br><br><br>

---

Return to Slides

---

<br><br><br><br>


## Using the Confidence Interval for Testing Hypotheses

**Null:** The average age of mothers in the population is 25 years; the random sample average is different due to chance.

**Alternative:** The average age of the mothers in the population is not 25 years.

Suppose you use the 5% cutoff for the p-value.

Based on the confidence interval, which hypothesis would you pick?

In [ ]:
bootstrap_means = bootstrapper(births, avg_maternal_age, 5000)
avg_maternal_age_ci = compute_ci(bootstrap_means, confidence_level=95)
avg_maternal_age_ci

In [ ]:
bootstrap_means = bootstrapper(births, avg_maternal_age, 5000)
avg_maternal_age_ci = compute_ci(bootstrap_means, confidence_level=99.9)
avg_maternal_age_ci

check it: https://www.cdc.gov/nchs/nsfg/key_statistics/b.htm

<br><br><br><br>

---

Return to Slides

---

<br><br><br><br>


## Average (Mean) 

There are many ways to compute the mean (average).

In [ ]:
values = make_array(2, 3, 3, 9)
values

In [ ]:
sum(values)/len(values)

In [ ]:
np.average(values)

In [ ]:
np.mean(values)

In [ ]:
(2 + 3 + 3 + 9)/4

Can also be framed as a weighted average of the unique values.

In [ ]:
2*(1/4) + 3*(2/4) + 9*(1/4)

In [ ]:
2*0.25 + 3*0.5 + 9*0.25

In [ ]:
values_table = Table().with_columns('value', values)
values_table

In [ ]:
bins_for_display = np.arange(0.5, 10.6, 1)
values_table.hist('value', bins = bins_for_display)

In [ ]:
## Make array of 10 2s, 20 3s, and 10 9s

new_vals = make_array(2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
                      3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
                      9, 9, 9, 9, 9, 9, 9, 9, 9, 9)

In [ ]:
Table().with_column('value', new_vals).hist(bins = bins_for_display)

In [ ]:
np.average(new_vals)

In [ ]:
Table().with_column('value', new_vals).hist(bins = bins_for_display)
plots.ylim(-0.04, 0.5)
plots.plot([0, 10], [0, 0], color='grey', lw=2)
plots.scatter(4.25, -0.015, marker='^', color='red', s=100)
plots.title('Average as a Center of Gravity');